In [ ]:
# read in pdf and extract text
# We want to ignore all the decorum and only extract the text (i.e. no page number etc.)
import pytesseract
#from PyPDF2 import PdfReader
from pypdf import PdfReader
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import fitz
from PIL import Image
import ftfy

import matplotlib.pyplot as plt

from io import StringIO
import re
import os

from tqdm import tqdm
from typing import List, Tuple

import signal

from __future__ import print_function
import sys
import threading
from time import sleep
import _thread as thread

In [ ]:
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'

In [ ]:
institute = 'Erasmus'
pdf_path = f'//Ds/data/LAB/laupodteam/AIOS/Bram/language_modeling/MEDICAL_TEXT/RAW/PRETRAINING/PhDTheses/{institute}'

In [ ]:
def quit_function(fn_name):
    # print to stderr, unbuffered in Python 2.
    print('{0} took too long'.format(fn_name), file=sys.stderr)
    sys.stderr.flush() # Python 3 stderr is likely buffered.
    raise SystemExit 
    #raise TimeoutError

def exit_after(s):
    '''
    use as decorator to exit process if 
    function takes longer than s seconds
    '''
    def outer(fn):
        def inner(*args, **kwargs):
            timer = threading.Timer(s, quit_function, args=[fn.__name__])
            timer.start()
            try:
                result = fn(*args, **kwargs)
            finally:
                timer.cancel()
            return result
        return inner
    return outer

In [ ]:
@exit_after(10)
def pdf_to_text(path, backends=['pypdf', 'fitz', 'pdfminer', 'pytesseract']) -> Tuple[List[str], str]:
    '''Extract text from pdf documents
        Source: https://towardsdatascience.com/pdf-preprocessing-with-python-19829752af9f
    '''
    def pdfminer(_path):
        manager = PDFResourceManager()
        retstr = StringIO()
        layout = LAParams(all_texts=False, detect_vertical=True)
        device = TextConverter(manager, retstr, laparams=layout)
        interpreter = PDFPageInterpreter(manager, device)
        with open(_path, 'rb') as filepath:
            for page in PDFPage.get_pages(filepath, check_extractable=True):
                interpreter.process_page(page)
        text = retstr.getvalue()
        device.close()
        retstr.close()
        return text.split('\n')

    def pypdfer(_path):
        bytes_stream = open(_path, 'rb')
        reader = PdfReader(bytes_stream, strict=True)
        return [p.extract_text(0) for p in reader.pages]
    
    error = ""

    if 'pypdf' in backends:
        try:
            return pypdfer(path), None
        except Exception as e_1:
            error = error + f"\n PyPDF failed: {e_1}"
   
    if 'fitz' in backends:
        try:
            pdf_file = fitz.open(path)
            pages = []
            for _p in pdf_file:
                pages.append(_p.getText())
            return pages, error
        except Exception as e_2:
            error = error + f"\n PyMuPDF failed: {e_2}"
        
    if 'pdfminer' in backends:
        try:
            return pdfminer(path), error
        except Exception as e_3:
            error = error + f"pdfminer failed: {e_3}"

    if 'pytesseract' in backends:
        try:
            return pytesseract.image_to_string(path, lang='en'), error
        except Exception as e_4:
            return error + f"\n PyTesseract failed: {e_4}" 
        
# https://arxiv.org/abs/2308.13418

In [ ]:
# We want to ignore 
# Title pages
# Table of contents
# Reference lists 
# Acknowledgements
# List of abbreviations
# List of figures

# Remove
# all empty lines or lines that only have numbers
# all pages with less than K words

# We want to extract
# body, summary(english), summary(dutch)

In [ ]:
re_numbers_at_start_of_sentence_plus = re.compile(r'(\d+\n\d*)')  # Matches numbers at the start of a sentence
re_numbers_at_start_of_sentence = re.compile(r'(\d+)\n')  # Matches numbers at the start of a sentence

re_numbers_at_start_of_string = re.compile(r'^(\d+)')  # Matches numbers at the start of a sentence
re_lines_with_only_numbers = re.compile(r'^\s*\d+\s*$', re.MULTILINE)  # Matches lines that contain only numbers
re_multiple_newlines = re.compile(r'\n+')
re_empty_lines = re.compile(r'\n\s*\n')
re_empty_lines_start = re.compile(r'^\s*\n')
re_empty_lines_end = re.compile(r'\n\s*$')
re_multiple_spaces = re.compile(r'\s+')

re_section_num = re.compile(r'^\d+\n(\d*)') 

In [ ]:
def extract_summary(Texts: List[str], max_scount: int=20) -> str:
    samenvatting = []
    capture = False
    scount = 0

    init_section_num =""
    for page in Texts:
        section_num = re_section_num.findall(page)        
        page = re_numbers_at_start_of_sentence_plus.sub(r'', page)  # Remove numbers at the start of a sentence
        if any((x in page.lower()[:60]) | (x in page.lower()[-60:]) for x in ['s amenvatting', 'samenvatting', 
                                                'nederlandse samenvatting', 
                                                'samenvatting in het nederlands',
                                                's amenvatting in het nederlands',     
                                                    'd utch summary',
                                                    'dutch summary',
                                                    'n ederlandse samenvatting']):
            capture = True
            init_section_num =  section_num
            scount += 1
        elif any((x in page.lower()[:60]) | (x in page.lower()[-60:])for x in ['d ankwoord', 
                                              'na woord',
                                              'a cknowledgment',
                                              'c ontents', 
                                              't able of contents', 
                                              'l ist of figures', 
                                              'l ist of abbreviations', 
                                              'a cknowledgements', 
                                              'r eferences',
                                              'dankwoord',
                                              'nawoord', 
                                              'acknowledgment',
                                              'contents', 
                                              'table of contents', 
                                              'list of figures', 
                                              'list of abbreviations', 
                                              'acknowledgements', 
                                              'references',
                                              's ummary', 
                                              'summary',
                                              'english summary']):
            capture = False
        elif section_num != init_section_num:
            capture = False
        
        if capture:
            scount += 1
            samenvatting.append(page)
        if scount >= max_scount:
            break
    summary = []
    capture = False
    scount = 0
    for page in Texts:
        page = re_numbers_at_start_of_sentence_plus.sub(r'', page)
        if any((x in page.lower()[:60]) | (x in page.lower()[-60:]) for x in ['s ummary', 'summary', 'english summary', 'summery']):
            capture = True
            init_section_num =  section_num
            scount += 1
        elif any((x in page.lower()[:60]) | (x in page.lower()[-60:]) for x in ['d ankwoord', 
                                              'na woord',
                                              'a cknowledgment',
                                              'c ontents', 
                                              't able of contents', 
                                              'l ist of figures', 
                                              'l ist of abbreviations', 
                                              'a cknowledgements', 
                                              'r eferences',
                                              'dankwoord',
                                              'nawoord', 
                                              'acknowledgment',
                                              'contents', 
                                              'table of contents', 
                                              'list of figures', 
                                              'list of abbreviations', 
                                              'acknowledgements', 
                                              'references',
                                              's amenvatting', 'samenvatting', 
                                              'nederlandse samenvatting', 
                                              'd utch summary',
                                              'dutch summary',
                                              'n ederlandse samenvatting']):
            capture = False
        elif section_num != init_section_num:
            capture = False
            
        if capture:
            scount += 1
            summary.append(page)
        if scount >= max_scount:
            break
    # remove numbers of the start of sentences
    # remove multiple newlines
    # remove empty lines
    # remove multiple spaces
    # remove lines with only numbers

    summary = [re_numbers_at_start_of_sentence.sub(r'', s) for s in summary]
    summary = [re_empty_lines_start.sub(r'', s) for s in summary]
    summary = [re_empty_lines_end.sub(r'', s) for s in summary]
    summary = [re_empty_lines.sub(r'', s) for s in summary]
    summary = [re_lines_with_only_numbers.sub(r'', s) for s in summary]
    summary = [re_numbers_at_start_of_string.sub(r'\n', s) for s in summary]

    samenvatting = [re_numbers_at_start_of_sentence.sub(r'', s) for s in samenvatting]
    samenvatting = [re_empty_lines_start.sub(r'', s) for s in samenvatting]
    samenvatting = [re_empty_lines_end.sub(r'', s) for s in samenvatting]
    samenvatting = [re_empty_lines.sub(r'', s) for s in samenvatting]
    samenvatting = [re_lines_with_only_numbers.sub(r'', s) for s in samenvatting]
    samenvatting = [re_numbers_at_start_of_string.sub(r'\n', s) for s in samenvatting]
    return '\n'.join(summary), '\n'.join(samenvatting)

In [ ]:
def extractor(Text: List[str], min_words: int=100) -> List[str]:
    Text = [ftfy.fix_text(t) for t in Text]

    Text = [t for t in Text if len(t.split())>50]
    Text = [re_numbers_at_start_of_sentence.sub('', t) for t in Text]
    Text = [re_numbers_at_start_of_string.sub('', t) for t in Text]
    Text = [re_lines_with_only_numbers.sub('', t) for t in Text]
    Text = [re_multiple_newlines.sub('\n', t) for t in Text]
    Text = [re_empty_lines.sub('\n', t) for t in Text]
    Text = [re_empty_lines_start.sub('', t) for t in Text]
    Text = [re_empty_lines_end.sub('', t) for t in Text]
    Text = [re_multiple_spaces.sub(' ', t) for t in Text]
    Text = [t for t in Text if len(t.split())>50]

    # ignore references
    reference_phrases = ['references', 'literature', 'bibliography', 'referenties', 'literatuurlijst']
    Text = [t for t in Text if not any(reference_phrase in t.lower() for reference_phrase in reference_phrases)]
    # ignore lime that start with numbers after the linebreak or have "doi:10" in them
    # scan the page for lines that start with numbers after a linebreak
    _TEXT = []
    for page in Text:
        lines = page.split('\n')
        __page= []
        for line in lines:
            if (not re.search(r'^\d+', line)) and ('doi:10' not in line.lower()):
                __page.append(line)
        _TEXT.append('\n'.join(__page))
    Text = _TEXT

    # ignore list of figures
    figure_phrases = ['list of figures', 'lijst van figuren']
    Text = [t for t in Text if not any(figure_phrase in t.lower() for figure_phrase in figure_phrases)]

    # ignore list of abbreviations
    abbreviation_phrases = ['list of abbreviations', 'lijst van afkortingen']
    Text = [t for t in Text if not any(abbreviation_phrase in t.lower() for abbreviation_phrase in abbreviation_phrases)]

    # ignore copyright page
    copyright_phrases = ['all rights reserved', 'no part of this publication may be reproduced', 'copyright', 'uitgeverij']
    Text = [t for t in Text if not any(copyright_phrase in t.lower() for copyright_phrase in copyright_phrases)]

    phd_phrases = ['volgens besluit van het college voor promoties', 'de graad van doctor aan']
    Text = [t for t in Text if not any(phd_phrase in t.lower() for phd_phrase in phd_phrases)]

    # ignore table of contents
    toc_phrases = ['inhoudsopgave', 'table of contents']
    Text = [t for t in Text if not any(toc_phrase in t.lower() for toc_phrase in toc_phrases)]
    # ignore if multiple sentences in a page start with "chapter \d"
    chapter_phrases = ['chapter ', 'hoofdstuk ']
    Text = [t for t in Text if sum(t.lower().count(chapter_phrase) for chapter_phrase in chapter_phrases)<2]

    # ignore acknowledgements
    acknowledgement_phrases = ['acknowledgements', 'acknowledgements', 'dankwoord', 'dankbetuiging']
    Text = [t for t in Text if not any(acknowledgement_phrase in t.lower() for acknowledgement_phrase in acknowledgement_phrases)]

    # ignore list of publications
    publication_phrases = ['list of publications', 'lijst van publicaties', 'bibliography', 'bibliografie']
    Text = [t for t in Text if not any(publication_phrase in t.lower() for publication_phrase in publication_phrases)]


    Text = [t for t in Text if len(t.split())>25]

    TextNumWords = [len(t.split()) for t in Text]
    

    return Text, TextNumWords

In [ ]:
Files = [f for f in os.listdir(pdf_path) if f.endswith('.pdf')]

ListOfTexts = []    
ListOfSummaries = []
ListOfNumWordLists = []

In [ ]:
for _File in tqdm(Files[len(ListOfNumWordLists):]):
    _path = os.path.join(pdf_path, _File)
    try:
        Text, error = pdf_to_text(_path)
    except Exception as e:
        print(f"Error processing {_File}: {e}")
        continue
    SummaryEnglish, SummaryDutch = extract_summary(Text)
    CleanedText, NumWordList = extractor(Text, min_words=25)

    for k, page in enumerate(CleanedText):
        ListOfTexts.append({
            'institute': institute,
            'file': _File,
            'pseudo_pagenum': k,
            'text': page
        })
    ListOfSummaries.append({
        'institute': institute,
        'file': _File,
        'summary_english': SummaryEnglish,
        'summary_dutch': SummaryDutch
    })
    ListOfNumWordLists.append({
        'institute': institute,
        'file': _File,
        'num_words_per_page': NumWordList
    })          
            

In [520]:
sum([_r for r in ListOfNumWordLists for _r in r['num_words_per_page']])

55168759

In [517]:
ListOfSummariesWith = [d for d in ListOfSummaries if (d['summary_english']!='') or (d['summary_dutch']!='')]
with_sum = len(ListOfSummariesWith)
ListOfSummariesWithout = [d for d in ListOfSummaries if (d['summary_english']=='') and (d['summary_dutch']=='')]
without_sum = len(ListOfSummariesWithout)
print(f'Out of {len(ListOfSummaries)} theses, {with_sum} have summaries and {without_sum} do not have summaries.')

Out of 2219 theses, 1153 have summaries and 1066 do not have summaries.


In [ ]:
import json

with open(f'//Ds/data/LAB/laupodteam/AIOS/Bram/language_modeling/MEDICAL_TEXT/RAW/PRETRAINING/PhDTheses/{institute}_texts.jsonl', 'w', encoding='utf-8') as f:
    for item in ListOfTexts:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

In [518]:
# the same for the summaries
with open(f'//Ds/data/LAB/laupodteam/AIOS/Bram/language_modeling/MEDICAL_TEXT/RAW/PRETRAINING/PhDTheses/{institute}_summaries.jsonl', 'w', encoding='utf-8') as f:
    for item in ListOfSummariesWith:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')